# Benchmark: Implement Levenshtein term similarity matrix and fast SCM between corpora ([RaRe-Technologies/gensim PR #2016][#2016])

 [#2016]: https://github.com/RaRe-Technologies/gensim/pull/2016 (Implement Levenshtein term similarity matrix and fast SCM between corpora - Pull Request #2016)

In [1]:
!git rev-parse HEAD

55a687c2d30bb9f4a03f4b73687ccc7803ca9993


In [2]:
from copy import deepcopy
from datetime import timedelta
from itertools import product
import logging
from math import floor, ceil, log10
import pickle
from random import sample, seed, shuffle
from time import time

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

def tqdm(iterable, total=None, desc=None):
    if total is None:
        total = len(iterable)
    for num_done, element in enumerate(tqdm_notebook(iterable, total=total)):
        logger.info("%s: %d / %d", desc, num_done, total)
        yield element

from gensim.corpora import Dictionary
import gensim.downloader as api
from gensim.similarities.index import AnnoyIndexer
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import UniformTermSimilarityIndex
from gensim.similarities import LevenshteinSimilarityIndex
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.utils import simple_preprocess

RANDOM_SEED = 12345

logger = logging.getLogger()
fhandler = logging.FileHandler(filename='matrix_speed.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)

pd.set_option('display.max_rows', None, 'display.max_seq_items', None)

In [3]:
"""Repeatedly run a benchmark callable given various configurations and
get a list of results.

Return a list of results of repeatedly running a benchmark callable.

Parameters
----------
benchmark : callable tuple -> dict
    A benchmark callable that accepts a configuration and returns results.
configurations : iterable of tuple
    An iterable of configurations that are used for calling the benchmark function.
results_filename : str
    A filename of a file that will be used to persistently store the results using
    pickle. If the file exists, then the function will load the stored results
    instead of calling the benchmark callable.

Returns
-------
iterable of tuple
    The return values of the individual invocations of the benchmark callable.

"""
def benchmark_results(benchmark, configurations, results_filename):
    try:
        with open(results_filename, "rb") as file:
            results = pickle.load(file)
    except IOError:
        configurations = list(configurations)
        shuffle(configurations)
        results = list(tqdm(
            (benchmark(configuration) for configuration in configurations),
            total=len(configurations), desc="benchmark"))
        with open(results_filename, "wb") as file:
            pickle.dump(results, file)
    return results

## Implement Levenshtein term similarity matrix

In Gensim PR [#1827][], we added a base implementation of the soft cosine measure (SCM). The base implementation would create term similarity matrices using a single complex procedure. In the Gensim PR [#2016][], we split the procedure into:

- **TermSimilarityIndex** builder classes that produce the $k$ most similar terms for a given term $t$ that are distinct from $t$ along with the term similarities, and
- the **SparseTermSimilarityMatrix** director class that constructs term similarity matrices and consumes term similarities produced by **TermSimilarityIndex** instances.

One of the benefits of this separation is that we can easily measure the speed at which a **TermSimilarityIndex** builder class produces term similarities and compare this speed with the speed at which the **SparseTermSimilarityMatrix** director class consumes term similarities. This allows us to see which of the classes are a bottleneck that slows down the construction of term similarity matrices.

In this notebook, we measure all the currently available builder and director classes. For the measurements, we use the [Google News word embeddings][word2vec-google-news-300] distributed with the C implementation of Word2Vec. From the word embeddings, we will derive a dictionary of 2.01M terms.

 [word2vec-google-news-300]: https://github.com/mmihaltz/word2vec-GoogleNews-vectors (word2vec-GoogleNews-vectors)
 [#1827]: https://github.com/RaRe-Technologies/gensim/pull/1827 (Implement Soft Cosine Measure - Pull Request #1827)
 [#2016]: https://github.com/RaRe-Technologies/gensim/pull/2016 (Implement Levenshtein term similarity matrix and fast SCM between corpora - Pull Request #2016)

In [4]:
full_model = api.load("word2vec-google-news-300")

try:
    full_dictionary = Dictionary.load("matrix_speed.dictionary")
except IOError:
    full_dictionary = Dictionary([[term] for term in full_model.vocab.keys()])
    full_dictionary.save("matrix_speed.dictionary")

### Director class benchmark
#### SparseTermSimilarityMatrix
First, we measure the speed at which the **SparseTermSimilarityMatrix** director class consumes term similarities.

In [5]:
def benchmark(configuration):
    dictionary, nonzero_limit, symmetric, repetition = configuration
    index = UniformTermSimilarityIndex(dictionary)
    
    start_time = time()
    matrix = SparseTermSimilarityMatrix(
        index, dictionary, nonzero_limit=nonzero_limit, symmetric=symmetric,
        dtype=np.float16).matrix
    end_time = time()
    
    duration = end_time - start_time
    return {
        "dictionary_size": len(dictionary),
        "nonzero_limit": nonzero_limit,
        "matrix_nonzero": matrix.nnz,
        "repetition": repetition,
        "symmetric": symmetric,
        "duration": duration, }

In [6]:
dictionary_sizes = [10**k for k in range(3, int(ceil(log10(len(full_dictionary)))))]
seed(RANDOM_SEED)
dictionaries = []
for size in tqdm(dictionary_sizes, desc="dictionaries"):
    dictionary = Dictionary([sample(list(full_dictionary.values()), size)])
    dictionaries.append(dictionary)
dictionaries.append(full_dictionary)
nonzero_limits = [1, 10, 100]
symmetry = (True, False)
repetitions = range(10)

configurations = product(dictionaries, nonzero_limits, symmetry, repetitions)
results = benchmark_results(benchmark, configurations, "matrix_speed.director_results")

The following tables show how long it takes to construct a term similarity matrix (the **duration** column), how many nonzero elements there are in the matrix (the **matrix_nonzero** column) and the mean term similarity consumption speed (the **consumption_speed** column) as we vary the dictionary size (the **dictionary_size** column) the maximum number of nonzero elements outside the diagonal in every column of the matrix (the **nonzero_limit** column), and the matrix symmetry constraint (the **symmetric** column). Ten independendent measurements were taken. The top table shows the mean values and the bottom table shows the standard deviations.

We can see that the symmetry constraint severely limits the number of nonzero elements in the resulting matrix. This in turn increases the consumption speed, since we end up throwing away most of the elements that we consume. The effects of the dictionary size on the mean term similarity consumption speed are minor.

In [7]:
df = pd.DataFrame(results)
df["consumption_speed"] = df.dictionary_size * df.nonzero_limit / df.duration
df = df.groupby(["dictionary_size", "nonzero_limit", "symmetric"])

def display(df):
    df["duration"] = [timedelta(0, duration) for duration in df["duration"]]
    df["matrix_nonzero"] = [int(nonzero) for nonzero in df["matrix_nonzero"]]
    df["consumption_speed"] = ["%.02f Kword pairs / s" % (speed / 1000) for speed in df["consumption_speed"]]
    return df

In [8]:
display(df.mean()).loc[
    [10000, len(full_dictionary)], :, :].loc[
    :, ["duration", "matrix_nonzero", "consumption_speed"]]

duration  matrix_nonzero  \
dictionary_size nonzero_limit symmetric                                   
10000           1             False     00:00:00.286091           20000   
                              True      00:00:00.166287           10002   
                10            False     00:00:01.573833          110000   
                              True      00:00:00.640328           10118   
                100           False     00:00:14.662728         1010000   
                              True      00:00:05.233251           20198   
2010000         1             False     00:01:02.938585         4020000   
                              True      00:00:35.977733         2010002   
                10            False     00:05:53.410117        22110000   
                              True      00:02:07.940066         2010118   
                100           False     01:06:08.946648       203010000   
                              True      00:17:06.064969         2020198   

                                              consumption_speed  
dictionary_size nonzero_limit symmetric                          
10000           1             False       34.95 Kword pairs / s  
                              True        60.15 Kword pairs / s  
                10            False       63.54 Kword pairs / s  
                              True       156.20 Kword pairs / s  
                100           False       68.20 Kword pairs / s  
                              True       191.09 Kword pairs / s  
2010000         1             False       31.94 Kword pairs / s  
                              True        55.87 Kword pairs / s  
                10            False       56.88 Kword pairs / s  
                              True       157.11 Kword pairs / s  
                100           False       50.65 Kword pairs / s  
                              True       195.91 Kword pairs / s

In [9]:
display(df.apply(lambda x: (x - x.mean()).std())).loc[
    [10000, len(full_dictionary)], :, :].loc[
    :, ["duration", "matrix_nonzero", "consumption_speed"]]

duration  matrix_nonzero  \
dictionary_size nonzero_limit symmetric                                   
10000           1             False     00:00:00.001519               0   
                              True      00:00:00.002255               0   
                10            False     00:00:00.013232               0   
                              True      00:00:00.009424               0   
                100           False     00:00:00.101245               0   
                              True      00:00:00.021103               0   
2010000         1             False     00:00:00.205360               0   
                              True      00:00:00.091344               0   
                10            False     00:00:01.252888               0   
                              True      00:00:00.302513               0   
                100           False     00:00:54.806356               0   
                              True      00:00:09.599669               0   

                                            consumption_speed  
dictionary_size nonzero_limit symmetric                        
10000           1             False      0.19 Kword pairs / s  
                              True       0.82 Kword pairs / s  
                10            False      0.53 Kword pairs / s  
                              True       2.27 Kword pairs / s  
                100           False      0.47 Kword pairs / s  
                              True       0.77 Kword pairs / s  
2010000         1             False      0.10 Kword pairs / s  
                              True       0.14 Kword pairs / s  
                10            False      0.20 Kword pairs / s  
                              True       0.37 Kword pairs / s  
                100           False      0.68 Kword pairs / s  
                              True       1.82 Kword pairs / s

### Builder class benchmark
#### UniformTermSimilarityIndex
First, we measure the speed at which the **UniformTermSimilarityIndex** builder class produces term similarities. **UniformTermSimilarityIndex** is a dummy class that just generates a sequence of constants. It produces much more term similarities per second than the **SparseTermSimilarityMatrix** is capable of consuming and its results will serve as an upper limit.

In [10]:
def benchmark(configuration):
    dictionary, nonzero_limit, repetition = configuration
    
    start_time = time()
    index = UniformTermSimilarityIndex(dictionary)
    end_time = time()
    constructor_duration = end_time - start_time
    
    start_time = time()
    for term in dictionary.values():
        for _j, _k in zip(index.most_similar(term, topn=nonzero_limit), range(nonzero_limit)):
            pass
    end_time = time()
    production_duration = end_time - start_time
    
    return {
        "dictionary_size": len(dictionary),
        "nonzero_limit": nonzero_limit,
        "repetition": repetition,
        "constructor_duration": constructor_duration,
        "production_duration": production_duration, }

In [11]:
nonzero_limits = [1, 10, 100, 1000]

configurations = product(dictionaries, nonzero_limits, repetitions)
results = benchmark_results(benchmark, configurations, "matrix_speed.builder_results.uniform")

The following tables show how long it takes to retrieve the most similar terms for all terms in a dictionary (the **production_duration** column) and the mean term similarity production speed (the **production_speed** column) as we vary the dictionary size (the **dictionary_size** column), and the maximum number of most similar terms that will be retrieved (the **nonzero_limit** column). Ten independendent measurements were taken. The top table shows the mean values and the bottom table shows the standard deviations.

The **production_speed** is proportional to **nonzero_limit**.

In [12]:
df = pd.DataFrame(results)
df["processing_speed"] = df.dictionary_size ** 2 / df.production_duration
df["production_speed"] = df.dictionary_size * df.nonzero_limit / df.production_duration
df = df.groupby(["dictionary_size", "nonzero_limit"])

def display(df):
    df["constructor_duration"] = [timedelta(0, duration) for duration in df["constructor_duration"]]
    df["production_duration"] = [timedelta(0, duration) for duration in df["production_duration"]]
    df["processing_speed"] = ["%.02f Kword pairs / s" % (speed / 1000) for speed in df["processing_speed"]]
    df["production_speed"] = ["%.02f Kword pairs / s" % (speed / 1000) for speed in df["production_speed"]]
    return df

In [13]:
display(df.mean()).loc[
    [1000, len(full_dictionary)], :, :].loc[
    :, ["production_duration", "production_speed"]]

production_duration         production_speed
dictionary_size nonzero_limit                                             
1000            1                 00:00:00.003828   261.66 Kword pairs / s
                10                00:00:00.009975  1002.80 Kword pairs / s
                100               00:00:00.073020  1372.84 Kword pairs / s
                1000              00:00:00.727086  1375.54 Kword pairs / s
2010000         1                 00:00:08.315807   241.71 Kword pairs / s
                10                00:00:21.027485   955.90 Kword pairs / s
                100               00:02:24.223933  1393.70 Kword pairs / s
                1000              00:23:57.702287  1398.09 Kword pairs / s

In [14]:
display(df.apply(lambda x: (x - x.mean()).std())).loc[
    [1000, len(full_dictionary)], :, :].loc[
    :, ["production_duration", "production_speed"]]

production_duration       production_speed
dictionary_size nonzero_limit                                           
1000            1                 00:00:00.000163  10.67 Kword pairs / s
                10                00:00:00.000174  17.08 Kword pairs / s
                100               00:00:00.004030  67.59 Kword pairs / s
                1000              00:00:00.009082  16.98 Kword pairs / s
2010000         1                 00:00:00.023885   0.70 Kword pairs / s
                10                00:00:00.054096   2.46 Kword pairs / s
                100               00:00:00.763313   7.42 Kword pairs / s
                1000              00:00:06.162980   5.99 Kword pairs / s

#### LevenshteinSimilarityIndex
Next, we measure the speed at which the **LevenshteinSimilarityIndex** builder class produces term similarities. **LevenshteinSimilarityIndex** is currently just a naïve implementation that produces much fewer term similarities per second than the **SparseTermSimilarityMatrix** class is capable of consuming.

In [15]:
def benchmark(configuration):
    dictionary, nonzero_limit, query_terms, repetition = configuration
    
    start_time = time()
    index = LevenshteinSimilarityIndex(dictionary)
    end_time = time()
    constructor_duration = end_time - start_time
    
    start_time = time()
    for term in query_terms:
        for _j, _k in zip(index.most_similar(term, topn=nonzero_limit), range(nonzero_limit)):
            pass
    end_time = time()
    production_duration = end_time - start_time
    
    return {
        "dictionary_size": len(dictionary),
        "mean_query_term_length": np.mean([len(term) for term in query_terms]),
        "nonzero_limit": nonzero_limit,
        "repetition": repetition,
        "constructor_duration": constructor_duration,
        "production_duration": production_duration, }

In [16]:
nonzero_limits = [1, 10, 100]
seed(RANDOM_SEED)
min_dictionary = sorted((len(dictionary), dictionary) for dictionary in dictionaries)[0][1]
query_terms = sample(list(min_dictionary.values()), 10)

configurations = product(dictionaries, nonzero_limits, [query_terms], repetitions)
results = benchmark_results(benchmark, configurations, "matrix_speed.builder_results.levenshtein")

The following tables show how long it takes to retrieve the most similar terms for ten randomly sampled terms from a dictionary (the **production_duration** column), the mean term similarity production speed (the **production_speed** column) and the mean term similarity processing speed (the **processing_speed** column) as we vary the dictionary size (the **dictionary_size** column), and the maximum number of most similar terms that will be retrieved (the **nonzero_limit** column). Ten independendent measurements were taken. The top table shows the mean values and the bottom table shows the standard deviations.

The **production_speed** is proportional to **nonzero_limit / dictionary_size**.

In [17]:
df = pd.DataFrame(results)
df["processing_speed"] = df.dictionary_size * len(query_terms) / df.production_duration
df["production_speed"] = df.nonzero_limit * len(query_terms) / df.production_duration
df = df.groupby(["dictionary_size", "nonzero_limit"])

def display(df):
    df["constructor_duration"] = [timedelta(0, duration) for duration in df["constructor_duration"]]
    df["production_duration"] = [timedelta(0, duration) for duration in df["production_duration"]]
    df["processing_speed"] = ["%.02f Kword pairs / s" % (speed / 1000) for speed in df["processing_speed"]]
    df["production_speed"] = ["%.02f word pairs / s" % speed for speed in df["production_speed"]]
    return df

In [18]:
display(df.mean()).loc[
    [1000, 1000000, len(full_dictionary)], :].loc[
    :, ["production_duration", "production_speed", "processing_speed"]]

production_duration         production_speed  \
dictionary_size nonzero_limit                                                
1000            1                 00:00:00.065093    153.95 word pairs / s   
                10                00:00:00.064572   1551.83 word pairs / s   
                100               00:00:00.067055  14932.14 word pairs / s   
1000000         1                 00:00:37.913246      0.26 word pairs / s   
                10                00:00:37.672419      2.66 word pairs / s   
                100               00:00:37.523020     26.66 word pairs / s   
2010000         1                 00:01:20.311723      0.12 word pairs / s   
                10                00:01:20.223581      1.25 word pairs / s   
                100               00:01:20.064439     12.49 word pairs / s   

                                     processing_speed  
dictionary_size nonzero_limit                          
1000            1              153.95 Kword pairs / s  
                10             155.18 Kword pairs / s  
                100            149.32 Kword pairs / s  
1000000         1              263.96 Kword pairs / s  
                10             265.62 Kword pairs / s  
                100            266.62 Kword pairs / s  
2010000         1              250.44 Kword pairs / s  
                10             250.73 Kword pairs / s  
                100            251.13 Kword pairs / s

In [19]:
display(df.apply(lambda x: (x - x.mean()).std())).loc[
    [1000, 1000000, len(full_dictionary)], :].loc[
    :, ["production_duration", "production_speed", "processing_speed"]]

production_duration       production_speed  \
dictionary_size nonzero_limit                                              
1000            1                 00:00:00.003128    7.43 word pairs / s   
                10                00:00:00.003058   74.56 word pairs / s   
                100               00:00:00.002477  572.64 word pairs / s   
1000000         1                 00:00:01.094520    0.01 word pairs / s   
                10                00:00:01.034344    0.07 word pairs / s   
                100               00:00:00.833463    0.58 word pairs / s   
2010000         1                 00:00:02.193144    0.00 word pairs / s   
                10                00:00:02.284553    0.04 word pairs / s   
                100               00:00:01.536381    0.24 word pairs / s   

                                   processing_speed  
dictionary_size nonzero_limit                        
1000            1              7.43 Kword pairs / s  
                10             7.46 Kword pairs / s  
                100            5.73 Kword pairs / s  
1000000         1              7.67 Kword pairs / s  
                10             7.23 Kword pairs / s  
                100            5.76 Kword pairs / s  
2010000         1              6.78 Kword pairs / s  
                10             7.07 Kword pairs / s  
                100            4.85 Kword pairs / s

#### WordEmbeddingSimilarityIndex
Lastly, we measure the speed at which the **WordEmbeddingSimilarityIndex** builder class constructs an instance and produces term similarities. Gensim currently supports slow and precise nearest neighbor search, and also approximate nearest neighbor search using [ANNOY][]. We evaluate both options.

 [ANNOY]: https://github.com/spotify/annoy (Approximate Nearest Neighbors in C++/Python optimized for memory usage and loading/saving to disk)

In [20]:
def benchmark(configuration):
    (model, dictionary), nonzero_limit, annoy_n_trees, query_terms, repetition = configuration
    use_annoy = annoy_n_trees > 0
    model.init_sims()
    
    start_time = time()
    if use_annoy:
        annoy = AnnoyIndexer(model, annoy_n_trees)
        kwargs = {"indexer": annoy}
    else:
        kwargs = {}
    index = WordEmbeddingSimilarityIndex(model, kwargs=kwargs)
    end_time = time()
    constructor_duration = end_time - start_time
    
    start_time = time()
    for term in query_terms:
        for _j, _k in zip(index.most_similar(term, topn=nonzero_limit), range(nonzero_limit)):
            pass
    end_time = time()
    production_duration = end_time - start_time
    
    return {
        "dictionary_size": len(dictionary),
        "mean_query_term_length": np.mean([len(term) for term in query_terms]),
        "nonzero_limit": nonzero_limit,
        "use_annoy": use_annoy,
        "annoy_n_trees": annoy_n_trees,
        "repetition": repetition,
        "constructor_duration": constructor_duration,
        "production_duration": production_duration, }

In [21]:
models = []
for dictionary in tqdm(dictionaries, desc="models"):
    if dictionary == full_dictionary:
        models.append(full_model)
        continue
    model = full_model.__class__(full_model.vector_size)
    model.vocab = {word: deepcopy(full_model.vocab[word]) for word in dictionary.values()}
    model.index2entity = []
    vector_indices = []
    for index, word in enumerate(full_model.index2entity):
        if word in model.vocab.keys():
            model.index2entity.append(word)
            model.vocab[word].index = len(vector_indices)
            vector_indices.append(index)
    model.vectors = full_model.vectors[vector_indices]
    models.append(model)
annoy_n_trees = [0] + [10**k for k in range(3)]
seed(RANDOM_SEED)
query_terms = sample(list(min_dictionary.values()), 1000)

configurations = product(zip(models, dictionaries), nonzero_limits, annoy_n_trees, [query_terms], repetitions)
results = benchmark_results(benchmark, configurations, "matrix_speed.builder_results.wordembeddings")

The following tables show how long it takes to construct an ANNOY index and the builder class instance (the **constructor_duration** column), how long it takes to retrieve the most similar terms for 1,000 randomly sampled terms from a dictionary (the **production_duration** column), the mean term similarity production speed (the **production_speed** column) and the mean term similarity processing speed (the **processing_speed** column) as we vary the dictionary size (the **dictionary_size** column), the maximum number of most similar terms that will be retrieved (the **nonzero_limit** column), and the number of constructed ANNOY trees (the **annoy_n_trees** column). Ten independendent measurements were taken. The top table shows the mean values and the bottom table shows the standard deviations.

If we do not use ANNOY (**annoy_n_trees**${}=0$), then **production_speed** is proportional to **nonzero_limit / dictionary_size**. 
If we do use ANNOY (**annoy_n_trees**${}>0$), then **production_speed** is proportional to **nonzero_limit / (annoy_n_trees)**${}^{1/2}$.

In [22]:
df = pd.DataFrame(results)
df["processing_speed"] = df.dictionary_size * len(query_terms) / df.production_duration
df["production_speed"] = df.nonzero_limit * len(query_terms) / df.production_duration
df = df.groupby(["dictionary_size", "nonzero_limit", "annoy_n_trees"])

def display(df):
    df["constructor_duration"] = [timedelta(0, duration) for duration in df["constructor_duration"]]
    df["production_duration"] = [timedelta(0, duration) for duration in df["production_duration"]]
    df["processing_speed"] = ["%.02f Kword pairs / s" % (speed / 1000) for speed in df["processing_speed"]]
    df["production_speed"] = ["%.02f Kword pairs / s" % (speed / 1000) for speed in df["production_speed"]]
    return df

In [23]:
display(df.mean()).loc[
    [1000000, len(full_dictionary)], [1, 100], [0, 1, 100]].loc[
    :, ["constructor_duration", "production_duration", "production_speed", "processing_speed"]]

constructor_duration  \
dictionary_size nonzero_limit annoy_n_trees                        
1000000         1             0                  00:00:00.000016   
                              1                  00:00:29.243768   
                              100                00:06:18.394023   
                100           0                  00:00:00.000014   
                              1                  00:00:29.327988   
                              100                00:06:17.906254   
2010000         1             0                  00:00:00.000013   
                              1                  00:01:30.093113   
                              100                00:23:21.211156   
                100           0                  00:00:00.000012   
                              1                  00:01:30.555084   
                              100                00:22:10.017924   

                                            production_duration  \
dictionary_size nonzero_limit annoy_n_trees                       
1000000         1             0                 00:00:19.814856   
                              1                 00:00:00.086151   
                              100               00:00:00.145153   
                100           0                 00:00:21.404975   
                              1                 00:00:00.148678   
                              100               00:00:01.267254   
2010000         1             0                 00:01:55.708445   
                              1                 00:00:00.169142   
                              100               00:00:00.317731   
                100           0                 00:02:12.106273   
                              1                 00:00:00.271253   
                              100               00:00:02.856382   

                                                   production_speed  \
dictionary_size nonzero_limit annoy_n_trees                           
1000000         1             0                0.05 Kword pairs / s   
                              1               11.61 Kword pairs / s   
                              100              6.89 Kword pairs / s   
                100           0                4.67 Kword pairs / s   
                              1              672.60 Kword pairs / s   
                              100             78.91 Kword pairs / s   
2010000         1             0                0.01 Kword pairs / s   
                              1                5.91 Kword pairs / s   
                              100              3.15 Kword pairs / s   
                100           0                0.76 Kword pairs / s   
                              1              368.69 Kword pairs / s   
                              100             35.01 Kword pairs / s   

                                                        processing_speed  
dictionary_size nonzero_limit annoy_n_trees                               
1000000         1             0                 50467.35 Kword pairs / s  
                              1              11607994.56 Kword pairs / s  
                              100             6889489.33 Kword pairs / s  
                100           0                 46718.15 Kword pairs / s  
                              1               6725972.71 Kword pairs / s  
                              100              789115.67 Kword pairs / s  
2010000         1             0                 17371.28 Kword pairs / s  
                              1              11883667.16 Kword pairs / s  
                              100             6341482.88 Kword pairs / s  
                100           0                 15215.26 Kword pairs / s  
                              1               7410628.89 Kword pairs / s  
                              100              703700.97 Kword pairs / s

In [24]:
display(df.apply(lambda x: (x - x.mean()).std())).loc[
    [1000000, len(full_dictionary)], [1, 100], [0, 1, 100]].loc[
    :, ["constructor_duration", "production_duration", "production_speed", "processing_speed"]]

constructor_duration  \
dictionary_size nonzero_limit annoy_n_trees                        
1000000         1             0                  00:00:00.000007   
                              1                  00:00:00.037389   
                              100                00:00:00.778346   
                100           0                         00:00:00   
                              1                  00:00:00.230572   
                              100                00:00:00.365938   
2010000         1             0                         00:00:00   
                              1                  00:00:00.054403   
                              100                00:02:36.227334   
                100           0                  00:00:00.000001   
                              1                  00:00:00.961168   
                              100                00:00:05.000325   

                                            production_duration  \
dictionary_size nonzero_limit annoy_n_trees                       
1000000         1             0                 00:00:00.038433   
                              1                 00:00:00.000601   
                              100               00:00:00.000842   
                100           0                 00:00:00.019706   
                              1                 00:00:00.000249   
                              100               00:00:00.004160   
2010000         1             0                 00:00:00.165553   
                              1                 00:00:00.000622   
                              100               00:00:00.017605   
                100           0                 00:00:00.546129   
                              1                 00:00:00.002506   
                              100               00:00:00.013226   

                                                 production_speed  \
dictionary_size nonzero_limit annoy_n_trees                         
1000000         1             0              0.00 Kword pairs / s   
                              1              0.08 Kword pairs / s   
                              100            0.04 Kword pairs / s   
                100           0              0.00 Kword pairs / s   
                              1              1.13 Kword pairs / s   
                              100            0.26 Kword pairs / s   
2010000         1             0              0.00 Kword pairs / s   
                              1              0.02 Kword pairs / s   
                              100            0.15 Kword pairs / s   
                100           0              0.00 Kword pairs / s   
                              1              3.35 Kword pairs / s   
                              100            0.16 Kword pairs / s   

                                                      processing_speed  
dictionary_size nonzero_limit annoy_n_trees                             
1000000         1             0                  97.76 Kword pairs / s  
                              1               79750.61 Kword pairs / s  
                              100             39420.33 Kword pairs / s  
                100           0                  43.04 Kword pairs / s  
                              1               11255.12 Kword pairs / s  
                              100              2582.72 Kword pairs / s  
2010000         1             0                  24.79 Kword pairs / s  
                              1               43763.82 Kword pairs / s  
                              100            308488.25 Kword pairs / s  
                100           0                  62.80 Kword pairs / s  
                              1               67336.24 Kword pairs / s  
                              100              3258.56 Kword pairs / s

## Implement fast SCM between corpora

In Gensim PR [#1827][], we added a base implementation of the soft cosine measure (SCM). The base implementation would compute SCM between single documents using the **softcossim** function. In the Gensim PR [#2016][], we intruduced the **SparseTermSimilarityMatrix.inner_product** method, which computes SCM not only between single documents, but also between a document and a corpus, and between two corpora.

For the measurements, we use the [Google News word embeddings][word2vec-google-news-300] distributed with the C implementation of Word2Vec. From the word embeddings, we will derive a dictionary of 2.01m terms. As a corpus, we will use a random sample of 100K articles from the 4.92m English [Wikipedia articles][enwiki].

 [word2vec-google-news-300]: https://github.com/mmihaltz/word2vec-GoogleNews-vectors (word2vec-GoogleNews-vectors)
 [enwiki]: https://github.com/RaRe-Technologies/gensim-data/releases/tag/wiki-english-20171001 (wiki-english-20171001)
 [#1827]: https://github.com/RaRe-Technologies/gensim/pull/1827 (Implement Soft Cosine Measure - Pull Request #1827)
 [#2016]: https://github.com/RaRe-Technologies/gensim/pull/2016 (Implement Levenshtein term similarity matrix and fast SCM between corpora - Pull Request #2016)

In [25]:
full_model = api.load("word2vec-google-news-300")

try:
    with open("matrix_speed.corpus", "rb") as file:
        full_corpus = pickle.load(file)        
except IOError:
    original_corpus = list(tqdm(api.load("wiki-english-20171001"), desc="original_corpus", total=4924894))
    seed(RANDOM_SEED)
    full_corpus = [
        simple_preprocess(u'\n'.join(article["section_texts"]))
        for article in tqdm(sample(original_corpus, 10**5), desc="full_corpus", total=10**5)]
    del original_corpus
    with open("matrix_speed.corpus", "wb") as file:
        pickle.dump(full_corpus, file)

try:
    full_dictionary = Dictionary.load("matrix_speed.dictionary")
except IOError:
    full_dictionary = Dictionary([[term] for term in full_model.vocab.keys()])
    full_dictionary.save("matrix_speed.dictionary")

### SCM between two documents
First, we measure the speed at which the **inner_product** method produces term similarities between single documents.

In [26]:
def benchmark(configuration):
    (matrix, dictionary, nonzero_limit), corpus, normalized, repetition = configuration
    corpus_size = len(corpus)
    corpus = [dictionary.doc2bow(doc) for doc in corpus]
    corpus = [vec for vec in corpus if len(vec) > 0]
    
    start_time = time()
    for vec1 in corpus:
        for vec2 in corpus:
            matrix.inner_product(vec1, vec2, normalized=normalized)
    end_time = time()
    duration = end_time - start_time
    
    return {
        "dictionary_size": matrix.matrix.shape[0],
        "matrix_nonzero": matrix.matrix.nnz,
        "nonzero_limit": nonzero_limit,
        "normalized": normalized,
        "corpus_size": corpus_size,
        "corpus_nonzero": len(corpus),
        "mean_document_length": np.mean([len(doc) for doc in corpus]),
        "repetition": repetition,
        "duration": duration, }

In [27]:
seed(RANDOM_SEED)
dictionary_sizes = [10**k for k in range(3, int(ceil(log10(len(full_dictionary)))))]
dictionaries = []
for size in tqdm(dictionary_sizes, desc="dictionaries"):
    dictionary = Dictionary([sample(list(full_dictionary.values()), size)])
    dictionaries.append(dictionary)
dictionaries.append(full_dictionary)
min_dictionary = sorted((len(dictionary), dictionary) for dictionary in dictionaries)[0][1]

corpus_sizes = [100, 1000]
corpora = []
for size in tqdm(corpus_sizes, desc="corpora"):
    corpus = sample(full_corpus, size)
    corpora.append(corpus)

models = []
for dictionary in tqdm(dictionaries, desc="models"):
    if dictionary == full_dictionary:
        models.append(full_model)
        continue
    model = full_model.__class__(full_model.vector_size)
    model.vocab = {word: deepcopy(full_model.vocab[word]) for word in dictionary.values()}
    model.index2entity = []
    vector_indices = []
    for index, word in enumerate(full_model.index2entity):
        if word in model.vocab.keys():
            model.index2entity.append(word)
            model.vocab[word].index = len(vector_indices)
            vector_indices.append(index)
    model.vectors = full_model.vectors[vector_indices]
    models.append(model)

nonzero_limits = [1, 10, 100]
matrices = []
for (model, dictionary), nonzero_limit in tqdm(
        list(product(zip(models, dictionaries), nonzero_limits)), desc="matrices"):
    annoy = AnnoyIndexer(model, 1)
    index = WordEmbeddingSimilarityIndex(model, kwargs={"indexer": annoy})
    matrix = SparseTermSimilarityMatrix(index, dictionary, nonzero_limit=nonzero_limit)
    matrices.append((matrix, dictionary, nonzero_limit))
    del annoy

normalization = (True, False)
repetitions = range(10)

/mnt/storage/home/novotny/gensim/gensim/matutils.py:738: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [28]:
configurations = product(matrices, corpora, normalization, repetitions)
results = benchmark_results(benchmark, configurations, "matrix_speed.inner-product_results.doc_doc")

The following tables show how long it takes to compute the **inner_product** method between all document vectors in a corpus (the **duration** column), how many nonzero document vectors there are in a corpus (the **corpus_nonzero** column), how many nonzero elements there are in a term similarity matrix (the **matrix_nonzero** column) and the mean document similarity production speed (the **speed** column) as we vary the dictionary size (the **dictionary_size** column), the size of the corpus (the **corpus_size** column), the maximum number of nonzero elements in a single column of the matrix (the **nonzero_limit** column), and the matrix symmetry constraint (the **symmetric** column). Ten independendent measurements were taken. The top table shows the mean values and the bottom table shows the standard deviations.

The **speed** is proportional to the square of the number of unique terms shared by the two document vectors. In our scenario as well as the standard IR scenario, this means **speed** is constant. Computing a normalized inner product (**normalized**${}={}$True) results in a constant speed decrease.

In [29]:
df = pd.DataFrame(results)
df["speed"] = df.corpus_nonzero**2 / df.duration
df = df.groupby(["dictionary_size", "corpus_size", "nonzero_limit", "normalized"])

def display(df):
    df["duration"] = [timedelta(0, duration) for duration in df["duration"]]
    df["speed"] = ["%.02f Kdoc pairs / s" % (speed / 1000) for speed in df["speed"]]
    return df

In [30]:
display(df.mean()).loc[
    [1000, 100000], :, [1, 100], :].loc[
    :, ["duration", "corpus_nonzero", "matrix_nonzero", "speed"]]

duration  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False      00:00:00.000844   
                                          True       00:00:00.000948   
                            100           False      00:00:00.000890   
                                          True       00:00:00.000852   
                1000        1             False      00:00:00.364683   
                                          True       00:00:00.401855   
                            100           False      00:00:00.370575   
                                          True       00:00:00.397365   
100000          100         1             False      00:00:06.010311   
                                          True       00:00:06.431314   
                            100           False      00:00:06.186657   
                                          True       00:00:06.708975   
                1000        1             False      00:10:13.258062   
                                          True       00:11:20.907826   
                            100           False      00:10:31.669595   
                                          True       00:11:31.505501   

                                                      corpus_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  1.0   
                                          True                   1.0   
                            100           False                  1.0   
                                          True                   1.0   
                1000        1             False                 24.0   
                                          True                  24.0   
                            100           False                 24.0   
                                          True                  24.0   
100000          100         1             False                 94.0   
                                          True                  94.0   
                            100           False                 94.0   
                                          True                  94.0   
                1000        1             False                931.0   
                                          True                 931.0   
                            100           False                931.0   
                                          True                 931.0   

                                                      matrix_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False               1000.0   
                                          True                1000.0   
                            100           False              84724.0   
                                          True               84724.0   
                1000        1             False               1000.0   
                                          True                1000.0   
                            100           False              84724.0   
                                          True               84724.0   
100000          100         1             False             102754.0   
                                          True              102754.0   
                            100           False            8134614.0   
                                          True             8134614.0   
                1000        1             False             102754.0   
                                          True              102754.0   
                            100           False            8134614.0   
                                          True             8134614.0   

                                                                    speed  
dictionary_size corpus_size nonzero_limit normalized                       
1000            1

In [31]:
display(df.apply(lambda x: (x - x.mean()).std())).loc[
    [1000, 100000], :, [1, 100], :].loc[
    :, ["duration", "corpus_nonzero", "matrix_nonzero", "speed"]]

duration  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False      00:00:00.000266   
                                          True       00:00:00.000233   
                            100           False      00:00:00.000231   
                                          True       00:00:00.000222   
                1000        1             False      00:00:00.012289   
                                          True       00:00:00.012813   
                            100           False      00:00:00.016282   
                                          True       00:00:00.008923   
100000          100         1             False      00:00:00.311038   
                                          True       00:00:00.199735   
                            100           False      00:00:00.371861   
                                          True       00:00:00.419363   
                1000        1             False      00:00:26.375716   
                                          True       00:00:37.038096   
                            100           False      00:00:27.173464   
                                          True       00:00:25.752512   

                                                      corpus_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
100000          100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   

                                                      matrix_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
100000          100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   

                                                                    speed  
dictionary_size corpus_size nonzero_limit normalized                       
1000            1

### SCM between a document and a corpus
Next, we measure the speed at which the **inner_product** method produces term similarities between documents and a corpus.

In [32]:
def benchmark(configuration):
    (matrix, dictionary, nonzero_limit), corpus, normalized, repetition = configuration
    corpus_size = len(corpus)
    corpus = [dictionary.doc2bow(doc) for doc in corpus if doc]
    
    start_time = time()
    for vec in corpus:
        matrix.inner_product(vec, corpus, normalized=normalized)
    end_time = time()
    duration = end_time - start_time
    
    return {
        "dictionary_size": matrix.matrix.shape[0],
        "matrix_nonzero": matrix.matrix.nnz,
        "nonzero_limit": nonzero_limit,
        "normalized": normalized,
        "corpus_size": corpus_size,
        "corpus_nonzero": len(corpus),
        "mean_document_length": np.mean([len(doc) for doc in corpus]),
        "repetition": repetition,
        "duration": duration, }

In [33]:
configurations = product(matrices, corpora, normalization, repetitions)
results = benchmark_results(benchmark, configurations, "matrix_speed.inner-product_results.doc_corpus")

The **speed** is inversely proportional to **matrix_nonzero**. Computing a normalized inner product (**normalized**${}={}$True) results in a constant speed decrease.

In [34]:
df = pd.DataFrame(results)
df["speed"] = df.corpus_nonzero**2 / df.duration
df = df.groupby(["dictionary_size", "corpus_size", "nonzero_limit", "normalized"])

def display(df):
    df["duration"] = [timedelta(0, duration) for duration in df["duration"]]
    df["speed"] = ["%.02f Kdoc pairs / s" % (speed / 1000) for speed in df["speed"]]
    return df

In [35]:
display(df.mean()).loc[
    [1000, 100000], :, [1, 100], :].loc[
    :, ["duration", "corpus_nonzero", "matrix_nonzero", "speed"]]

duration  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False      00:00:00.026415   
                                          True       00:00:00.031172   
                            100           False      00:00:00.049967   
                                          True       00:00:00.047319   
                1000        1             False      00:00:00.283383   
                                          True       00:00:00.304221   
                            100           False      00:00:00.623971   
                                          True       00:00:00.738579   
100000          100         1             False      00:00:00.418272   
                                          True       00:00:00.432476   
                            100           False      00:00:39.347374   
                                          True       00:00:37.148214   
                1000        1             False      00:00:11.689220   
                                          True       00:00:12.112989   
                            100           False      00:06:26.612026   
                                          True       00:06:45.335668   

                                                      corpus_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                100.0   
                                          True                 100.0   
                            100           False                100.0   
                                          True                 100.0   
                1000        1             False               1000.0   
                                          True                1000.0   
                            100           False               1000.0   
                                          True                1000.0   
100000          100         1             False                100.0   
                                          True                 100.0   
                            100           False                100.0   
                                          True                 100.0   
                1000        1             False               1000.0   
                                          True                1000.0   
                            100           False               1000.0   
                                          True                1000.0   

                                                      matrix_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False               1000.0   
                                          True                1000.0   
                            100           False              83340.0   
                                          True               83340.0   
                1000        1             False               1000.0   
                                          True                1000.0   
                            100           False              83340.0   
                                          True               83340.0   
100000          100         1             False             102420.0   
                                          True              102420.0   
                            100           False            8288496.0   
                                          True             8288496.0   
                1000        1             False             102420.0   
                                          True              102420.0   
                            100           False            8288496.0   
                                          True             8288496.0   

                                                                       speed  
dictionary_size corpus_size nonzero_limit normalized                          
1000       

In [36]:
display(df.apply(lambda x: (x - x.mean()).std())).loc[
    [1000, 100000], :, [1, 100], :].loc[
    :, ["duration", "corpus_nonzero", "matrix_nonzero", "speed"]]

duration  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False      00:00:00.005496   
                                          True       00:00:00.007937   
                            100           False      00:00:00.000453   
                                          True       00:00:00.007808   
                1000        1             False      00:00:00.009584   
                                          True       00:00:00.014151   
                            100           False      00:00:00.011108   
                                          True       00:00:00.011940   
100000          100         1             False      00:00:00.014609   
                                          True       00:00:00.015215   
                            100           False      00:00:04.042557   
                                          True       00:00:02.724963   
                1000        1             False      00:00:00.087808   
                                          True       00:00:00.318628   
                            100           False      00:00:37.485753   
                                          True       00:00:48.370403   

                                                      corpus_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
100000          100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   

                                                      matrix_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
100000          100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   

                                                                      speed  
dictionary_size corpus_size nonzero_limit normalized                         
1000         

### SCM between two corpora
Lastly, we measure the speed at which the **inner_product** method produces term similarities between entire corpora.

In [37]:
def benchmark(configuration):
    (matrix, dictionary, nonzero_limit), corpus, normalized, repetition = configuration
    corpus_size = len(corpus)
    corpus = [dictionary.doc2bow(doc) for doc in corpus]
    corpus = [vec for vec in corpus if len(vec) > 0]
    
    start_time = time()
    matrix.inner_product(corpus, corpus, normalized=normalized)
    end_time = time()
    duration = end_time - start_time
    
    return {
        "dictionary_size": matrix.matrix.shape[0],
        "matrix_nonzero": matrix.matrix.nnz,
        "nonzero_limit": nonzero_limit,
        "normalized": normalized,
        "corpus_size": corpus_size,
        "corpus_nonzero": len(corpus),
        "mean_document_length": np.mean([len(doc) for doc in corpus]),
        "repetition": repetition,
        "duration": duration, }

In [38]:
corpora.append(full_corpus)

configurations = product(matrices, corpora, normalization, repetitions)
results = benchmark_results(benchmark, configurations, "matrix_speed.inner-product_results.corpus_corpus")

The **speed** is inversely proportional to **matrix_nonzero**. Computing a normalized inner product (**normalized**${}={}$True) results in a constant speed decrease.

In [39]:
df = pd.DataFrame(results)
df["speed"] = df.corpus_nonzero**2 / df.duration
df = df.groupby(["dictionary_size", "corpus_size", "nonzero_limit", "normalized"])

def display(df):
    df["duration"] = [timedelta(0, duration) for duration in df["duration"]]
    df["speed"] = ["%.02f Kdoc pairs / s" % (speed / 1000) for speed in df["speed"]]
    return df

In [40]:
display(df.mean()).loc[
    [1000, 100000], :, [1, 100], :].loc[
    :, ["duration", "corpus_nonzero", "matrix_nonzero", "speed"]]

duration  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False      00:00:00.001417   
                                          True       00:00:00.004412   
                            100           False      00:00:00.002731   
                                          True       00:00:00.006891   
                1000        1             False      00:00:00.001284   
                                          True       00:00:00.004089   
                            100           False      00:00:00.002716   
                                          True       00:00:00.006884   
                100000      1             False      00:00:00.020794   
                                          True       00:00:00.026243   
                            100           False      00:00:00.033319   
                                          True       00:00:00.047398   
100000          100         1             False      00:00:00.002843   
                                          True       00:00:00.007447   
                            100           False      00:00:00.371558   
                                          True       00:00:01.055354   
                1000        1             False      00:00:00.010172   
                                          True       00:00:00.015548   
                            100           False      00:00:00.360084   
                                          True       00:00:01.114117   
                100000      1             False      00:00:33.289201   
                                          True       00:00:36.110679   
                            100           False      00:00:46.041435   
                                          True       00:00:52.112290   

                                                      corpus_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  3.0   
                                          True                   3.0   
                            100           False                  3.0   
                                          True                   3.0   
                1000        1             False                 31.0   
                                          True                  31.0   
                            100           False                 31.0   
                                          True                  31.0   
                100000      1             False               2876.0   
                                          True                2876.0   
                            100           False               2876.0   
                                          True                2876.0   
100000          100         1             False                 78.0   
                                          True                  78.0   
                            100           False                 78.0   
                                          True                  78.0   
                1000        1             False                828.0   
                                          True                 828.0   
                            100           False                828.0   
                                          True                 828.0   
                100000      1             False              84469.0   
                                          True               84469.0   
                            100           False              84469.0   
                                          True               84469.0   

                                                      matrix_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False               1000.0   
                                          True                1000.0   
                          

In [41]:
display(df.apply(lambda x: (x - x.mean()).std())).loc[
    [1000, 100000], :, [1, 100], :].loc[
    :, ["duration", "corpus_nonzero", "matrix_nonzero", "speed"]]

duration  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False      00:00:00.000271   
                                          True       00:00:00.000283   
                            100           False      00:00:00.000165   
                                          True       00:00:00.000552   
                1000        1             False      00:00:00.000053   
                                          True       00:00:00.000068   
                            100           False      00:00:00.000164   
                                          True       00:00:00.000147   
                100000      1             False      00:00:00.000904   
                                          True       00:00:00.000698   
                            100           False      00:00:00.002211   
                                          True       00:00:00.004056   
100000          100         1             False      00:00:00.000285   
                                          True       00:00:00.000351   
                            100           False      00:00:00.079347   
                                          True       00:00:00.049617   
                1000        1             False      00:00:00.000098   
                                          True       00:00:00.000237   
                            100           False      00:00:00.009330   
                                          True       00:00:00.094245   
                100000      1             False      00:00:00.299012   
                                          True       00:00:00.060703   
                            100           False      00:00:00.316624   
                                          True       00:00:00.533292   

                                                      corpus_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                100000      1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
100000          100         1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                1000        1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   
                100000      1             False                  0.0   
                                          True                   0.0   
                            100           False                  0.0   
                                          True                   0.0   

                                                      matrix_nonzero  \
dictionary_size corpus_size nonzero_limit normalized                   
1000            100         1             False                  0.0   
                                          True                   0.0   
                          